

---

---


                     *** DETECCIÓN DEL LÍMITE AGUA-TIERRA CON METODOLOGÍAS SEMI-AUTOMÁTICAS ***


---

---




---



# Ejecutar esta celda


## 2. Instalar módulos


In [2]:
import ee
import geemap
import ipywidgets as widgets
from datetime import datetime
import pandas as pd
import requests
from IPython.display import HTML, display
import os
from zmq.constants import NULL

## 3. Mapa base




In [6]:
ee.Initialize()# Inicializar Earth Engine

Map = geemap.Map(center=(-40, -64), zoom=4) #crear el mapa inetrcativo con coordenadas especificas
Map.add_basemap('HYBRID')

## 4. Código de interfaz visual

In [7]:
#..................................................... Cartas
cartas = ee.FeatureCollection("projects/e-tenglerevelyn/assets/Cartas_50000")

# Obtener los valores únicos de la columna "carac"
valoresCarac = cartas.aggregate_array("carac").distinct().getInfo()

# Crear etiquetas
etiqueta = widgets.Label(value='Seleccione una carta 1:50.000')
etiqueta_fecha = widgets.Label(value='Elija el período de las imágenes')
etiqueta_vacia = widgets.Label(value='__________________________________________________________________________________________')

# Crear una lista desplegable con los registros
lista_registros = widgets.Select(options=valoresCarac, description='Opciones')

# Definir una función para manejar el evento de cambio de selección
def on_value_change(change):
    valor_carta = change['new']
    geometria_seleccionada = cartas.filter(ee.Filter.eq("carac", valor_carta))

    Map.addLayer(geometria_seleccionada, {}, 'Geometría seleccionada')

# Registrar la función de manejo del evento de cambio
lista_registros.observe(on_value_change, names='value')

#..................................................... Período de tiempo
date_picker_inicio = widgets.DatePicker(
    description='Fecha inicial:',
    value=datetime.now().date(),  # Establecer la fecha actual como valor inicial
    disabled=False
)

date_picker_fin = widgets.DatePicker(
    description='Fecha final:',
    value=datetime.now().date(),  # Establecer la fecha actual como valor inicial
    disabled=False
)

hbox_fecha= widgets.HBox([date_picker_inicio, date_picker_fin])

#..............................................................

opcion_area = widgets.RadioButtons(
    options=['Dibujar polígono en el mapa', 'Utilizar carta 1:50.000'],
    description='Seleccionar forma de determinar el área: ',
    layout=widgets.Layout(width='50%')
)

## 5. Entrenamiento

### Muestras preestablecidas del entrenamiento

In [8]:
# MUESTRAS
muestras = ee.FeatureCollection("projects/ee-gonzalodgign/assets/AguaTierra/LineadeCosta/Muestras_Argentina_Borrador")

# ZONA DE ENTRENAMIENTO
zona_muestreo = ee.FeatureCollection("projects/ee-tenglerevelyn/assets/AREA_A-T")

### Colección de imágenes del entrenamiento


In [12]:
# Colección de imágenes Sentinel 2
s2 = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate('2023-01-01', '2023-03-31') \
    .filterBounds(zona_muestreo) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)

s2c_ = s2.reduce(ee.Reducer.mean())
s2c = s2c_.clip(zona_muestreo)

### Indices espectrales

In [13]:
# NDWI
NDWI = s2c.normalizedDifference(['B3_mean', 'B8_mean']).rename('NDWImean')

# Stack
stackIslas_a = NDWI

### Entrenamiento

In [14]:
muestras_1 = muestras.randomColumn('random')

# División de muestras
entrenamiento = muestras_1.filter(ee.Filter.lt('random', 0.7))
validacion = muestras_1.filter(ee.Filter.gt('random', 0.7))

# Entrenar el clasificador utilizando datos de muestras
training = stackIslas_a.sampleRegions(
  collection=entrenamiento,
  properties=['class', 'random'],
  scale=30
)

# Bandas a utilizar en la clasificación
bandas_sel = ['NDWImean']

# Entrenar clasificador RF
classifier = ee.Classifier.minimumDistance().train(
    features = training,
    classProperty = 'class',
    inputProperties = bandas_sel
)

## 6. Clasificación

In [16]:
classified_b = None
extencion = None

def obtenerFecha():
    # Obtener las fechas seleccionadas
    fecha_inicio = date_picker_inicio.value
    fecha_fin = date_picker_fin.value

    # Convertir las fechas al formato deseado ('YYYY-MM-DD')
    fecha_inicio_str = fecha_inicio.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')

    # Convertir las fechas en objetos ee.Date
    fecha_inicio_ee = ee.Date(fecha_inicio_str)
    fecha_fin_ee = ee.Date(fecha_fin_str)

    # Utilizar las fechas convertidas en el parámetro time_interval
    time_interval = (fecha_inicio_ee, fecha_fin_ee)

    # Retornar el resultado
    return time_interval


def area_extencion():
  global extencion
    ####....................................................................... formas de elegir un área de interés

  selected_value = opcion_area.value
  print("Valor seleccionado:", selected_value)

  if (selected_value == 'Dibujar polígono en el mapa'):
  # ..................................................Región DIBUJADA
    Map.draw_features
    if (Map.draw_last_feature != None):
      extencion = ee.FeatureCollection(Map.draw_last_feature)#Roi Bounding Box (guarda en variable el bbox)
    #print("ok")
  else:
  # ..................................................Región CARTA
    valor_carta = lista_registros.value
    geometria_seleccionada = cartas.filter(ee.Filter.eq("carac", valor_carta))
    extencion = ee.FeatureCollection(geometria_seleccionada)

  return extencion


In [17]:
def clasificacion_agua_tierra(b):

  global classified_b  # Indica que deseas utilizar la variable global
####..............................................................................................................
  extencion = area_extencion()

# .................................................. Colección de imágenes Sentinel 2
  s2_b = ee.ImageCollection("COPERNICUS/S2") \
      .filterDate(*obtenerFecha()) \
      .filterBounds(extencion) \
      .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)

  s2c_b = s2_b.reduce(ee.Reducer.mean())
  s2c_b = s2c_b.clip(extencion)

# NDWI
  NDWI_b = s2c_b.normalizedDifference(['B3_mean', 'B8_mean']).rename('NDWImean')

# Stack
  stackIslas_b = NDWI_b

# Clasificar la imagen
  classified_b = stackIslas_b.classify(classifier)

# Añadir capa clasificada al mapa
  Map.addLayer(classified_b, {'min': 1, 'max': 2, 'palette': N1Color}, 'Clasificado')
  print("Se ha cargado en el mapa el resultado de la clasificación")

In [18]:
#----------  Botón  ----------#
# Establece el ancho del botón
button_width = '100px'  # Puedes ajustar el valor según tus necesidades

submit = widgets.Button(
    description='Clasificar',
    button_style='primary',
    tooltip='Click en el botón para detectar agua-tierra',
    layout=widgets.Layout(width=button_width)
)

output_s = widgets.Output()

#si se presiona el botón se llama a la función
submit.on_click(clasificacion_agua_tierra)

## 7. Estilo

In [19]:
# Color de categorías Agua/NoAgua
N1Color = [
    '#0000ff',  # Corriente de agua - clase 1
    '#764C04'  # Tierra/NoAgua - clase 2
]

imageVisParam2 = {
    'bands': ["B3_mean", "B2_mean", "B1_mean"],
    'gamma': 1,
    'max': 2789.728888888889,
    'min': 1875.3822222222223,
    'opacity': 1
}

## 8. Descarga Raster *


In [20]:
def descarga_raster(b):
  #print(f'Procesando descarga Raster...')
  global classified_b  # Indica que deseas utilizar la variable global
  global extencion

  url = classified_b.getDownloadURL({
      'name': 'LimiteAguaTierra',
      'scale': 10,
      'region': extencion.geometry()
  })

 #print(f'Ingresar al siguiente link para descargar el archivo resultante en formato Raster: {url}')

In [21]:
# Establece el ancho del botón
button_width_2 = '200px'  # Puedes ajustar el valor según tus necesidades

boton_descarga_raster = widgets.Button(
    description='Descargar Raster',
    button_style='primary',
    tooltip='Click en el botón para descargar el resultado en formato raster',
    layout=widgets.Layout(width=button_width_2)
)

#si se presiona el botón se llama a la función
boton_descarga_raster.on_click(descarga_raster)



## 7. Vectorización y descarga *



### Vectorizar

In [22]:
def descarga_vectorial(b):
    # Deshabilita el botón mientras se procesa la exportación
    submit.disabled = True

    # Muestra un mensaje de progreso
    with output:
        print('Procesando descarga Vectorial...')

    global classified_b
    global extencion

    vectores = classified_b.reduceToVectors(
        reducer=ee.Reducer.countEvery(),
        scale=10,
        geometry=extencion
    )

    Agua = ee.FeatureCollection(vectores)

    descripcion = 'Linea_de_Agua'
    formato = 'SHP'

    exportacion = ee.batch.Export.table.toDrive(
        collection=Agua,
        description=descripcion,
        folder='',
        fileNamePrefix=descripcion,
        fileFormat=formato
    )

    # Iniciar la exportación
    exportacion.start()

    status = exportacion.status()

    while status['state'] in ['READY', 'RUNNING']:
        # Realiza alguna acción mientras la exportación está en progreso
        pass

    if status['state'] == 'COMPLETED':
        with output:
            print('El archivo ha sido descargado en Drive. Búscalo como Linea_de_Agua.')

    # Habilita el botón nuevamente
    submit.disabled = False

### Descargar archivo vectorizado

In [23]:
#---------------------------------------------
boton_descarga_vectorial = widgets.Button(
    description='Descargar Vectorial',
    button_style='primary',
    tooltip='Click en el botón para descargar el resultado en formato vectorial',
    layout=widgets.Layout(width=button_width_2)
)

output = widgets.Output()

#si se presiona el botón se llama a la función
boton_descarga_vectorial.on_click(descarga_vectorial)

# Interfaz

In [24]:
Map

Map(center=[-40, -64], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [25]:
widgets.VBox([etiqueta_fecha,hbox_fecha, etiqueta_vacia, opcion_area, etiqueta, lista_registros, etiqueta_vacia, submit])

In [26]:
#widgets.VBox([boton_descarga_raster, boton_descarga_vectorial])